In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("xiyan123/idc-regular-ps50-idx5")

print("Path to dataset files:", path)

100%|██████████| 1.54G/1.54G [00:19<00:00, 87.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/xiyan123/idc-regular-ps50-idx5/versions/1


In [9]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

IMG_SIZE = 50
data = []
labels = []

dataset_path = path   # FIXED HERE

limit = 15000
count = 0

for patient in os.listdir(dataset_path):

    patient_path = os.path.join(dataset_path, patient)

    for label in ["0", "1"]:

        folder = os.path.join(patient_path, label)

        if not os.path.exists(folder):
            continue

        for img_name in os.listdir(folder):

            if count >= limit:
                break

            img_path = os.path.join(folder, img_name)

            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            data.append(img)
            labels.append(int(label))

            count += 1

        if count >= limit:
            break

    if count >= limit:
        break

data = np.array(data)
labels = np.array(labels)

print("Loaded images:", len(data))


Loaded images: 15000


In [11]:
print(data.shape)
print(labels.shape)


(15000, 50, 50, 3)
(15000,)


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (12000, 50, 50, 3)
Test shape: (3000, 50, 50, 3)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Flatten images
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_flat, y_train)
print("LR Accuracy:", accuracy_score(y_test, lr.predict(X_test_flat)))

# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_flat, y_train)
print("KNN Accuracy:", accuracy_score(y_test, knn.predict(X_test_flat)))

# SVM
svm = SVC()
svm.fit(X_train_flat, y_train)
print("SVM Accuracy:", accuracy_score(y_test, svm.predict(X_test_flat)))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR Accuracy: 0.7326666666666667
KNN Accuracy: 0.776
SVM Accuracy: 0.8156666666666667


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout

model1 = Sequential()

model1.add(Conv2D(32, (3,3), activation='relu', input_shape=(50,50,3)))
model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(MaxPooling2D((2,2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(2, activation='softmax'))

model1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model1.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.6272 - loss: 1.1453 - val_accuracy: 0.7267 - val_loss: 0.5436
Epoch 2/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.7568 - loss: 0.5099 - val_accuracy: 0.7650 - val_loss: 0.4707
Epoch 3/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.7648 - loss: 0.4886 - val_accuracy: 0.7596 - val_loss: 0.4753
Epoch 4/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.7724 - loss: 0.4725 - val_accuracy: 0.7875 - val_loss: 0.4733
Epoch 5/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.7870 - loss: 0.4652 - val_accuracy: 0.7800 - val_loss: 0.4612
Epoch 6/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.7818 - loss: 0.4645 - val_accuracy: 0.7858 - val_loss: 0.4631
Epoch 7/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.7867 - loss: 0.4606 - val_accuracy: 0.7946 - val_loss: 0.4453
Epoch 8/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.7976 - loss: 0.4402 - val_accuracy: 0.7962 - val_

In [15]:
from tensorflow.keras.layers import BatchNormalization

model2 = Sequential()

model2.add(Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(50,50,3)))
model2.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model2.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2,activation='softmax'))

model2.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


Epoch 1/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.7328 - loss: 1.3457 - val_accuracy: 0.3275 - val_loss: 2.5964
Epoch 2/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.8124 - loss: 0.4376 - val_accuracy: 0.3275 - val_loss: 5.9531
Epoch 3/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.8328 - loss: 0.3872 - val_accuracy: 0.3275 - val_loss: 6.6794
Epoch 4/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.8383 - loss: 0.3847 - val_accuracy: 0.3300 - val_loss: 6.6378
Epoch 5/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.8435 - loss: 0.3692 - val_accuracy: 0.3329 - val_loss: 4.9182
Epoch 6/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.8556 - loss: 0.3371 - val_accuracy: 0.3812 - val_loss: 1.7807
Epoch 7/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.8622 - loss: 0.3355 - val_accuracy: 0.6696 - val_loss: 1.5070
Epoch 8/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.8722 - loss: 0.3099 - val_accuracy: 0.6992 - v

In [16]:
model3 = Sequential()

model3.add(Conv2D(32,(5,5),activation='relu',input_shape=(50,50,3)))
model3.add(MaxPooling2D((3,3)))

model3.add(Conv2D(32,(5,5),activation='relu'))
model3.add(MaxPooling2D((3,3)))

model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(2,activation='softmax'))

model3.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model3.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


Epoch 1/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 28s 349ms/step - accuracy: 0.6504 - loss: 0.6307 - val_accuracy: 0.7300 - val_loss: 0.5574
Epoch 2/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 341ms/step - accuracy: 0.7249 - loss: 0.5540 - val_accuracy: 0.7337 - val_loss: 0.5211
Epoch 3/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 41s 341ms/step - accuracy: 0.7535 - loss: 0.5232 - val_accuracy: 0.7400 - val_loss: 0.5177
Epoch 4/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 25s 339ms/step - accuracy: 0.7650 - loss: 0.5098 - val_accuracy: 0.7708 - val_loss: 0.4749
Epoch 5/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 352ms/step - accuracy: 0.7782 - loss: 0.4778 - val_accuracy: 0.7579 - val_loss: 0.4959
Epoch 6/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 338ms/step - accuracy: 0.7711 - loss: 0.4990 - val_accuracy: 0.7862 - val_loss: 0.4709
Epoch 7/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 42s 352ms/step - accuracy: 0.7732 - loss: 0.4925 - val_accuracy: 0.7621 - val_loss: 0.4886
Epoch 8/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 41s 349ms/step - accuracy: 0.7841 - loss: 0.4764 - val_accu

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model3.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))


94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step
              precision    recall  f1-score   support

           0       0.87      0.74      0.80      2021
           1       0.59      0.77      0.67       979

    accuracy                           0.75      3000
   macro avg       0.73      0.76      0.74      3000
weighted avg       0.78      0.75      0.76      3000

[[1497  524]
 [ 221  758]]
